# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *

In [3]:
results_dir = 'results/new-models/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation()

/Users/bencottier/Projects/training_cost_2/training-cost-trends/cost.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontier_pcd_df['Training compute (FLOP)'] = pd.to_numeric(frontier_pcd_df['Training compute (FLOP)'], errors='coerce')


In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(68, 5508, 69)

# Cost estimation

In [6]:
# Drop Gemini Ultra
# frontier_pcd_df = frontier_pcd_df[frontier_pcd_df['System'] != 'Gemini Ultra']
# frontier_pcd_df = frontier_pcd_df[frontier_pcd_df['System'] != 'GPT-4']

In [7]:
cost_df = estimate_costs(frontier_pcd_df, hardware_df, price_df, impute_pcd_data=False)

==== System: Yuan 1.0 ====
Training time is required but no value found

==== System: OpenAI TI7 DOTA 1v1 ====
Training time is required but no value found

==== System: Claude 2 ====
Training time is required but no value found

==== System: Turing-NLG ====
Training time is required but no value found

==== System: GPT-3.5 (text-davinci-003) ====
Training time is required but no value found

==== System: AlphaGo Master ====
Training time is required but no value found

==== System: Minerva (540B) ====
Trying Google TPU v4 at 2022-04-01 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Found price: 1.45

==== System: FTW ====
Training time is required but no value found

==== System: AlphaGo Zero ====
Trying Google TPU v1 at 2017-07-30 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Could not find price
==== Sys

In [8]:
cost_df

,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,Publication date,...,Training cloud compute vendor,Training data center,Archived links,Batch size,Batch size notes,Datasets,Organization categorization,System,Training time (chip hours),Cost
System,,,,,,,,,,,,,,,,,,,,,
Yuan 1.0,Language,Language modelling,"Shaohua Wu, Xudong Zhao, Tong Yu, Rongguo Zhan...",SOTA Improvement,"""The zero-shot average scores of both LM and P...",NaN,https://arxiv.org/abs/2110.04725,37.0,Yuan 1.0: Large-Scale Pre-trained Language Mod...,2021-10-12,...,NaN,NaN,NaN,6881280.0,"Table 2. Batch size 3360, sequence length 2048...",NaN,Industry,Yuan 1.0,NaN,NaN
OpenAI TI7 DOTA 1v1,Games,DOTA,NaN,"SOTA Improvement,Historical significance",NaN,NaN,https://openai.com/research/dota-2,0.0,Dota 2,2017-08-11,...,NaN,NaN,NaN,NaN,NaN,NaN,Industry,OpenAI TI7 DOTA 1v1,NaN,NaN
Claude 2,Language,Language modelling,NaN,Historical significance,NaN,API accessible,"https://www.anthropic.com/index/claude-2, http...",0.0,NaN,2023-07-11,...,NaN,NaN,NaN,NaN,NaN,NaN,Industry,Claude 2,NaN,NaN
Turing-NLG,Language,Text autocompletion,Corby Rosset,SOTA Improvement,"from paper: ""Turing Natural Language Generatio...",NaN,https://www.microsoft.com/en-us/research/blog/...,114.0,Turing-NLG: A 17-billion-parameter language mo...,2020-02-13,...,NaN,NaN,NaN,NaN,NaN,NaN,Industry,Turing-NLG,NaN,NaN
GPT-3.5 (text-davinci-003),Language,Language modelling,NaN,"SOTA Improvement,Historical significance,Signi...",NaN,API accessible,https://platform.openai.com/docs/models/gpt-3-5,NaN,NaN,2022-11-28,...,NaN,NaN,NaN,NaN,NaN,NaN,Industry,GPT-3.5 (text-davinci-003),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HyperClova,Language,NaN,"Boseop Kim, HyoungSeok Kim, Sang-Woo Lee, Gich...",SOTA Improvement,"""HyperCLOVA with our training configuration sh...",NaN,https://arxiv.org/abs/2109.04650,79.0,What Changes Can Large-scale Language Models B...,2021-09-10,...,NaN,NaN,NaN,NaN,"""All models use the mini-batch size of 1,024""....",NaN,"Industry,Industry",HyperClova,658636.8,948436.992
PanGu-Σ,Language,"Code generation,Language modelling","Xiaozhe Ren, Pingyi Zhou, Xinfan Meng, Xinjing...",SOTA Improvement,"""Our experimental findings show that PanGu-{\S...",NaN,https://arxiv.org/abs/2303.10845,28.0,PanGu-Σ: Towards Trillion Parameter Language M...,2023-03-20,...,NaN,NaN,NaN,524288.0,"""We train PanGu-Σ with global batch size of 51...",NaN,Industry,PanGu-Σ,1228800.0,NaN
GLM-130B,Language,NaN,NaN,SOTA Improvement,"""GLM-130B achieves an accuracy of 80.2% on zer...",NaN,https://keg.cs.tsinghua.edu.cn/glm-130b/posts/...,60.0,GLM-130B: An open bilingual pre-trained model,2022-08-04,...,NaN,NaN,NaN,NaN,NaN,NaN,Academia,GLM-130B,1105920.0,1603584.000


In [9]:
cost_df['Cost'].notna().sum()

32

In [10]:
system = 'AlphaGo Fan'
print('Cost:', cost_df.loc[system, 'Cost'])
print('Training hardware:', cost_df.loc[system, 'Training hardware'])
print('Training time (hours):', cost_df.loc[system, 'Training time (hours)'])
print('Hardware quantity:', cost_df.loc[system, 'Hardware quantity'])
print('Hardware utilization:', cost_df.loc[system, 'Hardware utilization'])

Cost: nan
Training hardware: nan
Training time (hours): nan
Hardware quantity: nan
Hardware utilization: nan


# Apply inflation adjustment

In [11]:
# TODO move to function

In [12]:
cost_df['Publication date']

System
Yuan 1.0                     2021-10-12
OpenAI TI7 DOTA 1v1          2017-08-11
Claude 2                     2023-07-11
Turing-NLG                   2020-02-13
GPT-3.5 (text-davinci-003)   2022-11-28
                                ...    
HyperClova                   2021-09-10
PanGu-Σ                      2023-03-20
GLM-130B                     2022-08-04
Flamingo                     2022-04-29
GPT-NeoX-20B                 2022-02-09
Name: Publication date, Length: 68, dtype: datetime64[ns]

In [13]:
from_year_month = cost_df['Publication date'].apply(str)
cost_df['Publication date'] = from_year_month

In [14]:
cost_df['Publication date']

System
Yuan 1.0                      2021-10-12 00:00:00
OpenAI TI7 DOTA 1v1           2017-08-11 00:00:00
Claude 2                      2023-07-11 00:00:00
Turing-NLG                    2020-02-13 00:00:00
GPT-3.5 (text-davinci-003)    2022-11-28 00:00:00
                                     ...         
HyperClova                    2021-09-10 00:00:00
PanGu-Σ                       2023-03-20 00:00:00
GLM-130B                      2022-08-04 00:00:00
Flamingo                      2022-04-29 00:00:00
GPT-NeoX-20B                  2022-02-09 00:00:00
Name: Publication date, Length: 68, dtype: object

In [15]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [16]:
cost_df['Cost (inflation-adjusted)'].dropna()

System
Minerva (540B)              1.347019e+07
PaLM (540B)                 1.238906e+07
OPT-175B                    1.493975e+06
ALIGN                       1.655001e+05
GPT-4                       8.353709e+07
GLaM                        2.093016e+06
AlphaStar                   3.889027e+05
RoBERTa Large               1.600176e+05
GNMT                        1.947392e+05
GShard (dense)              9.691116e+05
Megatron-LM (8.3B)          2.160704e+05
Gopher (280B)               3.499808e+06
Meena                       6.971103e+05
IMPALA                      1.381309e+02
Falcon 180B                 2.581632e+07
JFT                         3.344073e+04
Megatron-Turing NLG 530B    6.405653e+06
LaMDA                       1.319586e+06
T5-11B                      2.366316e+05
BigGAN-deep 512x512         2.647313e+04
LLaMA-65B                   1.410426e+06
Switch                      6.197414e+05
Gemini Ultra                1.914000e+08
GPT-3 175B (davinci)        4.511300e+06
BERT-Larg

In [17]:
cost_df['Cost (inflation-adjusted)'].notna().sum()

32

# Regression

# Plots

In [18]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (USD, nominal)')

# title
fig.update_layout(title_text='Cost of cloud compute to train frontier ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [19]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text='Cost of cloud compute to train frontier ML systems')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

# Export data

In [20]:
keep_cols = [
    'Domain',
    'Task',
    'Open-source',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,Domain,Task,Open-source,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),Epochs,...,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Training time (chip hours),Cost,Cost (inflation-adjusted)
System,,,,,,,,,,,,,,,,,,,,,
Yuan 1.0,Language,Language modelling,NaN,Yuan 1.0: Large-Scale Pre-trained Language Mod...,2021-10-12 00:00:00,Inspur,2.457300e+11,3.538000e+23,1.000000e+12,0.22,...,China,NaN,NaN,2128.0,0.450,NaN,NaN,NaN,NaN,NaN
OpenAI TI7 DOTA 1v1,Games,DOTA,NaN,Dota 2,2017-08-11 00:00:00,OpenAI,NaN,6.046095e+20,NaN,NaN,...,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Claude 2,Language,Language modelling,API accessible,NaN,2023-07-11 00:00:00,Anthropic,NaN,3.866000e+24,NaN,NaN,...,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Turing-NLG,Language,Text autocompletion,NaN,Turing-NLG: A 17-billion-parameter language mo...,2020-02-13 00:00:00,Microsoft,1.700000e+10,1.570000e+22,3.480000e+10,3.39,...,Multinational,NaN,NaN,256.0,NaN,NaN,NaN,NaN,NaN,NaN
GPT-3.5 (text-davinci-003),Language,Language modelling,API accessible,NaN,2022-11-28 00:00:00,OpenAI,NaN,2.578000e+24,NaN,NaN,...,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HyperClova,Language,NaN,NaN,What Changes Can Large-scale Language Models B...,2021-09-10 00:00:00,"NAVER,Search Solutions",8.200000e+10,1.476000e+23,1.900000e+11,NaN,...,"Korea (Republic of),Korea (Republic of)",NaN,NaN,1024.0,0.200,NaN,NaN,658636.8,948436.992,9.801649e+05
PanGu-Σ,Language,"Code generation,Language modelling",NaN,PanGu-Σ: Towards Trillion Parameter Language M...,2023-03-20 00:00:00,Huawei Noah's Ark Lab,1.085000e+12,4.670000e+23,2.467500e+11,1.84,...,China,NaN,NaN,512.0,NaN,NaN,NaN,1228800.0,NaN,NaN
GLM-130B,Language,NaN,NaN,GLM-130B: An open bilingual pre-trained model,2022-08-04 00:00:00,Tsinghua University,1.300000e+11,3.778000e+23,NaN,1.00,...,China,NaN,NaN,768.0,0.433,NaN,NaN,1105920.0,1603584.000,1.635573e+06


In [21]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv')